# Cloud performance variability prediction#

In [5]:
import csv
import pandas as pd
import numpy as np

In [6]:
disk_info = []
with open ('./cloudlab-benchmarks-osdi18-data-and-nodebooks/data/raw-data/disk_info.csv') as f:
    for i in f:
        i = i.strip('\n')
        i = i.split(',')
        disk_info.append(i)

In [7]:
disk_info = pd.DataFrame(disk_info[1:], columns = disk_info[0])
disk_info

,run_uuid,timestamp,nodeid,nodeuuid,disk_name,disk_model,disk_serial,disk_size,npartitions,disk_type
0,d29989cd-c414-4af5-b04f-f29561b0240d,1495271912,ms0225,68285417-72da-11e4-85c8-179a28bd1b8c,/dev/sda,XR0120GEBLT,14230C9CD4C4,111.8G,3,SSD
1,a3d37542-7bf9-468f-a630-700715c9b1ff,1495272018,clnode123,cedf924b-fee2-11e5-ba07-000099989701,/dev/sdb,ST91000640NS,9XG9FW2C,931.5G,0,HDD
2,a3d37542-7bf9-468f-a630-700715c9b1ff,1495272018,clnode123,cedf924b-fee2-11e5-ba07-000099989701,/dev/sda,ST91000640NS,9XG9MB8Q,931.5G,4,HDD
3,368d9894-c37f-49de-a073-3cab47f69a3a,1495272078,clnode102,f2537c2f-fee1-11e5-ba07-000099989701,/dev/sdb,ST91000640NS,9XG9MA8Z,931.5G,0,HDD
4,368d9894-c37f-49de-a073-3cab47f69a3a,1495272078,clnode102,f2537c2f-fee1-11e5-ba07-000099989701,/dev/sda,ST91000640NS,9XG9L6T1,931.5G,4,HDD
...,...,...,...,...,...,...,...,...,...,...
17704,db427808-47f3-4599-9a24-3973c4425788,1522865516,ms0134,466b2b8c-72d5-11e4-85c8-179a28bd1b8c,/dev/sda,XR0120GEBLT,14230C9CDEEE,111.8G,3,SSD
17705,d2731139-52e3-4697-a0bc-a8a6beab0255,1522865516,ms0107,43cc585a-72d5-11e4-85c8-179a28bd1b8c,/dev/sda,XR0120GEBLT,14230C9CDED0,111.8G,3,SSD
17706,b1389312-a648-4e87-9046-085644802b02,1522865512,ms0839,cd9b0d15-8fbf-11e6-8c08-a53a3a50acf8,/dev/nvme0n1,THNSN5256GPU7 TOSHIBA,563S1002TUFV,238.5G,4,SSD
17707,b23f3655-2293-4be7-8cd0-be41ab0357be,1522865513,ms0802,e218b7a2-8f4b-11e6-8c08-a53a3a50acf8,/dev/nvme0n1,THNSN5256GPU7 TOSHIBA,563S1021TUFV,238.5G,4,SSD


In [9]:
disk_results = []
with open ('./cloudlab-benchmarks-osdi18-data-and-nodebooks/data/raw-data/disk_results.csv') as f:
    for i in f:
        i = i.strip('\n')
        i = i.split(',')
        disk_results.append(i)

In [10]:
disk_results = pd.DataFrame(disk_results[1:], columns = disk_results[0])
disk_results

,run_uuid,timestamp,nodeid,nodeuuid,testname,runtime,size,max,min,mean,stdev,units,device,iodepth
0,d29989cd-c414-4af5-b04f-f29561b0240d,1495271912,ms0225,68285417-72da-11e4-85c8-179a28bd1b8c,randwrite,720.002,1402100.0,2024.0,1832.0,1949.05,11.1741,KB/s,/dev/sda2,1
1,d29989cd-c414-4af5-b04f-f29561b0240d,1495271912,ms0225,68285417-72da-11e4-85c8-179a28bd1b8c,randread,105.565,10485800.0,172912.0,7.0,98894.2,62899.6,KB/s,/dev/sda2,4096
2,d29989cd-c414-4af5-b04f-f29561b0240d,1495271912,ms0225,68285417-72da-11e4-85c8-179a28bd1b8c,randwrite,720.283,1413080.0,2023.0,0.0,1961.17,57.8824,KB/s,/dev/sda2,4096
3,d29989cd-c414-4af5-b04f-f29561b0240d,1495271912,ms0225,68285417-72da-11e4-85c8-179a28bd1b8c,write,720.002,1402050.0,2011.0,1896.0,1948.92,10.3035,KB/s,/dev/sda2,1
4,d29989cd-c414-4af5-b04f-f29561b0240d,1495271912,ms0225,68285417-72da-11e4-85c8-179a28bd1b8c,randread,140.652,10485800.0,102976.0,25928.0,74530.0,36564.8,KB/s,/dev/sda2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141403,faf52960-818e-48e1-88d0-2bf58bfa0771,1522865528,ms0238,66ea8663-72da-11e4-85c8-179a28bd1b8c,write,720.001,1412670.0,2000.0,1896.0,1963.71,10.3456,KB/s,/dev/sda2,1
141404,faf52960-818e-48e1-88d0-2bf58bfa0771,1522865528,ms0238,66ea8663-72da-11e4-85c8-179a28bd1b8c,randread,140.03,10485800.0,103976.0,25672.0,74884.6,37212.3,KB/s,/dev/sda2,1
141405,faf52960-818e-48e1-88d0-2bf58bfa0771,1522865528,ms0238,66ea8663-72da-11e4-85c8-179a28bd1b8c,read,55.047,10485800.0,195185.0,182440.0,190613.0,2078.31,KB/s,/dev/sda2,4096
141406,faf52960-818e-48e1-88d0-2bf58bfa0771,1522865528,ms0238,66ea8663-72da-11e4-85c8-179a28bd1b8c,read,138.682,10485800.0,77088.0,72904.0,75617.2,584.633,KB/s,/dev/sda2,1
